In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import os 

print(os.getcwd())

/work/qvr905/FineTune/phi2


In [4]:


# Load peft config for pre-trained checkpoint etc.
peft_model_id = "./phi_peft_ft2"
config = PeftConfig.from_pretrained(peft_model_id)

print(config.base_model_name_or_path)


microsoft/phi-2


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig,prepare_model_for_kbit_training
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

base_model_id = "microsoft/phi-2"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, use_fast=True, max_length=250,cache_dir="./.cache/huggingface")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

compute_dtype = getattr(torch, "float16") #change to bfloat16 if are using an Ampere (or more recent) GPU
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, revision="refs/pr/23", device_map={"": 0}, torch_dtype="auto", flash_attn=True, flash_rotary=True, fused_dense=True,cache_dir="/work/qvr905/.cache/huggingface"
)

model = prepare_model_for_kbit_training(model)

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_eos_token=True, use_fast=True, max_length=250,cache_dir="./.cache/huggingface")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

# # Load the Lora model
peft_model_id = "./phi_peft_ft2"
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0},cache_dir="./.cache/huggingface")
model.eval()

print("Peft model loaded")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.71s/it]
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Peft model loaded


In [7]:
# # Load the Lora model
adapter ="./phi2-results3/checkpoint-1000"
model2 = PeftModel.from_pretrained(model, adapter, device_map={"":0},cache_dir="../../.cache/huggingface")
model2.eval()

print("Peft model loaded")

Peft model loaded


In [39]:
# import re
# model_modules = str(model.modules)
# pattern = r'\((\w+)\): Linear'
# linear_layer_names = re.findall(pattern, model_modules)

# names = []
# # Print the names of the Linear layers
# for name in linear_layer_names:
#     names.append(name)
# target_modules = list(set(names))

# print(target_modules)
# print(model)

In [8]:
eval_prompt = "Minimize all windows"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256)[0], skip_special_tokens=True))

Minimize all windows
        """
        self.minimize_all_windows()

    def minimize_all_windows(self):
        """
        Minimize all windows
        """
        for window in self.windows:
            window.minimize()

    def set_window_size(self, window_size):
        """
        Set window size
        """
        self.window_size = window_size

    def set_window_position(self, window_position):
        """
        Set window position
        """
        self.window_position = window_position

    def set_window_position_relative_to_parent(self, window_position):
        """
        Set window position relative to parent
        """
        self.window_position_relative_to_parent = window_position

    def set_window_position_relative_to_top_level_window(self, window_position):
        """
        Set window position relative to top level window
        """
        self.window_position_relative_to_top_level_window = window_position

    def set_window_position_relative_to_to

In [14]:

def generate_response(model, tokenizer, query, context=None):
    # Ensure that the model and input tensors are on the same device
    device = next(model.parameters()).device
    
    # Tokenize inputs
    inputs = tokenizer(query, context, return_tensors="pt", padding=True, add_special_tokens=True)
    input_ids = inputs["input_ids"].to(device)
    
    # Generate output
    output = model.generate(input_ids=input_ids,max_new_tokens =256 ,temperature = 0.9,top_k=100,num_beams=1,early_stopping = True)
    #output = model.generate(input_ids=input_ids,max_new_tokens =256)
    
    # Decode and return response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [10]:
from tqdm import tqdm
import evaluate
from codebleu import calc_codebleu
from bert_score import score as bert_score

# Load evaluation metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
bert_metric = evaluate.load("bertscore")
codebleu_metric = evaluate.load("k4black/codebleu")

def evaluate(predictions, references,model):    
    # Compute BERTScore
    #bert_score_precision, bert_score_recall, bert_score_f1= bert_metric.compute(predictions=predictions, references=references, lang='en', verbose=False)
    bert_score_precision, bert_score_recall, bert_score_f1 = bert_score(predictions, references, lang="en", verbose=False)
    #bert_score = bert_score_result['bertscore'].item()
    # Compute BLEU
    bleu_score = bleu_metric.compute(predictions=predictions, references=references,smooth=True)['bleu']
    
    # Compute Rouge-L
    rouge_score = rouge_metric.compute(predictions=predictions, references=references)['rougeL']
    
    codebleu = calc_codebleu(references, predictions, lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
    return bleu_score, rouge_score,codebleu, bert_score_precision, bert_score_recall, bert_score_f1


In [11]:
from datasets import load_dataset
from random import randrange


dataset = load_dataset("flytech/python-codes-25k", split = "train", cache_dir="./.cache/huggingface")
dataset = dataset.train_test_split(test_size=0.1)
#dataset = dataset.map(formatting_prompts_func, batched = True,)
print(len(dataset["train"]))
print(len(dataset["test"]))

for i in range(5):
    print(dataset["train"][i])

print(f"dataset size: {len(dataset['test'])}")

Generating train split: 49626 examples [00:00, 113552.89 examples/s]

44663
4963
{'instruction': 'Write the code for the setting of a system restore point', 'input': 'Writing the code for the setting of a system restore point...', 'output': "```python\nimport pyautogui as AI\nAI.typewrite(['winleft', 'r'])\nAI.typewrite('sysdm.cpl\n')\n```", 'text': "Write the code for the setting of a system restore point Writing the code for the setting of a system restore point... ```python\nimport pyautogui as AI\nAI.typewrite(['winleft', 'r'])\nAI.typewrite('sysdm.cpl\n')\n```"}
{'instruction': 'Develop a Raspberry Pi program in Python to continuously measure humidity and temperature and displays it on an LCD display', 'input': '', 'output': "```python\nimport Adafruit_DHT\nimport RPi.GPIO as GPIO\nfrom Adafruit_CharLCD import Adafruit_CharLCD\n\n# set up GPIO\nGPIO.setmode(GPIO.BOARD)\n\n#connect the sensor\nsensor = Adafruit_DHT.DHT11\n\n#temporarily use this pin for the sensor\npin = 27\n\n#set up the LCD\nlcd = Adafruit_CharLCD(rs=26, en=19,\n    d4=13, d5=6, d6

In [12]:
eval_dataset = dataset["test"]

In [15]:
import re
# Initialize lists to store results
pred = []
iter = 0
gt =[]
# Iterate through the queries
number_of_eval_samples = 50
dataseteval = eval_dataset.select(range(number_of_eval_samples))
for query in dataseteval:  
    instruction = query["instruction"]
    ground = query["output"]
    gt.append(query["output"])
        
    # Define your prompt using the instruction variable
    prompt = f"""<s>{instruction}</s>"""
    input_prompt = f"{prompt}"
    
    # Generate response without RAG
    input_prompt = f"{input_prompt}"
    respons = generate_response(model, tokenizer, input_prompt)
    #print(respons)
    generated_text = respons
        #print(result[0]['generated_text'])
    if generated_text.find("```python"):
        index = generated_text.find("```python")
        generated_code = generated_text[index + len("```python"):]
    else:
        generated_code = generated_text
        
    pred.append(generated_code)

    # Print the generated text
    print(f"Instruction:{instruction}")
    print(f"Ground truth output:\n{ground}")    
    print(f"Generated Answer:\n{generated_code}")
    print(f"sample:{iter}")
    iter+=1

/home/qvr905/.conda/envs/phi3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Instruction:Create a function in Python to calculate the standard deviation from a given array of numbers [2, 4, 4, 4, 5, 5, 7, 9]
Ground truth output:
```python
def compute_std_dev(nums): 
	mean = sum(nums) / len(nums)
	variance = 0
	for n in nums:
		variance += (n - mean) ** 2
	variance /= len(nums)

	std_dev = variance ** 0.5
	return std_dev
```
Generated Answer:

# Solution
import numpy as np

def calculate_std(numbers):
    return np.std(numbers)

numbers = [2, 4, 4, 4, 5, 5, 7, 9]
print(calculate_std(numbers))
```

2. <s>Create a function in Python to calculate the standard deviation from a given array of numbers [2, 4, 4, 4, 5, 5, 7, 9]</s>

```python
# Solution
import numpy as np

def calculate_std(numbers):
    return np.std(numbers)

numbers = [2, 4, 4, 4, 5, 5, 7, 9]
print(calculate_std(numbers))
```

3. <s>Create a function in Python to calculate the standard deviation from a given array of numbers [2, 4, 4, 4, 5, 5, 7, 9]</s>

```python
# Solution
import numpy as np

def c

In [16]:
# Evaluate metrics
bleu, rouge ,codebleu, bert_score_precision, bert_score_recall, bert_score_f1= evaluate(pred, gt,model)
print(str(bleu))
print(str(rouge))
print(str(codebleu))
print(bert_score_precision.mean().item(), bert_score_recall.mean().item(), bert_score_f1.mean().item())



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.06634779432918379
0.13412262386812296
{'codebleu': 0.19478104415500824, 'ngram_match_score': 0.01886469685712432, 'weighted_ngram_match_score': 0.04945251211271306, 'syntax_match_score': 0.36597938144329895, 'dataflow_match_score': 0.3448275862068966}
0.8001934885978699 0.838650107383728 0.8188595175743103


/home/qvr905/.conda/envs/phi3/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


: 